In [3]:
from global_dce import global_dce
from local_dce import local_value_numbering_function
from local_value_numbering import local_value_numbering_function

In [6]:
#!/usr/bin/env python3

import sys
import json

# Bril code examples as strings

# Example 1: A program that can be optimized effectively
bril_program_optimized = """
{
  "functions": [
    {
      "name": "main",
      "instrs": [
        {"dest": "a", "op": "const", "type": "int", "value": 1},
        {"dest": "b", "op": "const", "type": "int", "value": 2},
        {"dest": "c", "op": "add", "type": "int", "args": ["a", "b"]},
        {"dest": "d", "op": "add", "type": "int", "args": ["a", "b"]},
        {"dest": "e", "op": "add", "type": "int", "args": ["c", "d"]},
        {"op": "print", "args": ["e"]}
      ]
    }
  ]
}
"""

# Example 2: A program that can't be optimized with the current passes but can be optimized manually
bril_program_not_optimized = """
{
  "functions": [
    {
      "name": "main",
      "instrs": [
        {"dest": "n", "op": "const", "type": "int", "value": 5},
        {"dest": "i", "op": "const", "type": "int", "value": 0},
        {"dest": "sum", "op": "const", "type": "int", "value": 0},
        {"label": "loop"},
        {"dest": "lt", "op": "lt", "type": "bool", "args": ["i", "n"]},
        {"op": "br", "args": ["lt"], "labels": ["end", "loop_body"]},
        {"label": "end"},
        {"op": "print", "args": ["sum"]},
        {"op": "jmp", "labels": ["exit"]},
        {"label": "loop_body"},
        {"dest": "a", "op": "const", "type": "int", "value": 1},
        {"dest": "sum", "op": "add", "type": "int", "args": ["sum", "a"]},
        {"dest": "const_1", "op": "const", "type": "int", "value": 1},
        {"dest": "i", "op": "add", "type": "int", "args": ["i", "const_1"]},
        {"op": "jmp", "labels": ["loop"]},
        {"label": "exit"}
      ]
    }
  ]
}
"""

def optimize_program(prog):
    # Apply Global Dead Code Elimination
    prog = global_dce(prog)
    
    # Apply Local Dead Code Elimination to each function
    for func in prog['functions']:
        local_value_numbering_function(func)
    
    # Apply Local Value Numbering to each function
    for func in prog['functions']:
        local_value_numbering_function(func)
    
    return prog

if __name__ == "__main__":
    prog_json = bril_program_optimized
    prog = json.loads(prog_json)
    optimized_prog = optimize_program(prog)
    print(json.dumps(optimized_prog, indent=2))
    
    
    prog_json = bril_program_not_optimized
    prog = json.loads(prog_json)
    optimized_prog = optimize_program(prog)
    print(json.dumps(optimized_prog, indent=2))


{
  "functions": [
    {
      "name": "main",
      "instrs": [
        {
          "dest": "a",
          "op": "const",
          "type": "int",
          "value": 1
        },
        {
          "dest": "b",
          "op": "const",
          "type": "int",
          "value": 2
        },
        {
          "dest": "c",
          "op": "add",
          "type": "int",
          "args": [
            "a",
            "b"
          ]
        },
        {
          "op": "id",
          "dest": "d",
          "type": "int",
          "args": [
            "c"
          ]
        },
        {
          "dest": "e",
          "op": "add",
          "type": "int",
          "args": [
            "c",
            "d"
          ]
        },
        {
          "op": "print",
          "args": [
            "e"
          ]
        }
      ]
    }
  ]
}
{
  "functions": [
    {
      "name": "main",
      "instrs": [
        {
          "dest": "n",
          "op": "const",
          "type"